In [143]:
import pandas as pd

In [144]:
input_file = r'C:\Users\user\Desktop\YMM-2\Data\Input\G60 WI.xlsx'
output_file = r'..\Data\Output\Schema G60.xlsx'

In [145]:
df_input = pd.read_excel(input_file)
df_input.columns

Index([               'Famille',                     'WP',
       'Production Module MA15',                   'Step',
               'Component Name',             'Unnamed: 5',
                  'SAP NO MA15',                   'Note',
                           'CS',                 'Colour',
               'From Connector',            'From Cavity',
                 'To Connector',              'To Cavity',
                  'Unnamed: 14',                    'SPS',
                  'Unnamed: 16',            'Unnamed: 17',
            2025-03-01 00:00:00,            'Unnamed: 19',
                  'Unnamed: 20',            'Unnamed: 21',
                  'Unnamed: 22',            'Unnamed: 23',
                  'Unnamed: 24',            'Unnamed: 25',
                  'Unnamed: 26',            'Unnamed: 27',
                  'Unnamed: 28',            'Unnamed: 29',
                  'Unnamed: 30',            'Unnamed: 31',
                  'Unnamed: 32',            'Unnamed: 33

In [146]:
df_input['SPS'].dtypes

dtype('O')

In [147]:
df_input['SPS'].isnull().sum()

np.int64(299)

In [148]:
all_sheets = list(df_input['SPS'].unique())

In [149]:
sps_data = df_input[df_input['SPS'] == 'SA36-A']

In [150]:
PM_sps = list(sps_data['Production Module MA15'].unique())

In [151]:
df_input.columns

Index([               'Famille',                     'WP',
       'Production Module MA15',                   'Step',
               'Component Name',             'Unnamed: 5',
                  'SAP NO MA15',                   'Note',
                           'CS',                 'Colour',
               'From Connector',            'From Cavity',
                 'To Connector',              'To Cavity',
                  'Unnamed: 14',                    'SPS',
                  'Unnamed: 16',            'Unnamed: 17',
            2025-03-01 00:00:00,            'Unnamed: 19',
                  'Unnamed: 20',            'Unnamed: 21',
                  'Unnamed: 22',            'Unnamed: 23',
                  'Unnamed: 24',            'Unnamed: 25',
                  'Unnamed: 26',            'Unnamed: 27',
                  'Unnamed: 28',            'Unnamed: 29',
                  'Unnamed: 30',            'Unnamed: 31',
                  'Unnamed: 32',            'Unnamed: 33

In [152]:
# import xlwings as xw
# import pandas as pd
# import numpy as np


    
# with pd.ExcelWriter('multi_feuilles.xlsx', engine='xlsxwriter') as writer:      

#   for sheet in all_sheets:
#       if pd.isna(sheet):
#           continue
#       sheet_name = wb.sheets.add(name=sheet)
      
#       sps_data = df_input[df_input['SPS'] == sheet]
      
#       arrays = [np.array(sps_data['Production Module MA15']),
#                 np.array(sps_data['Step']),
#               #   np.array(sps_data['Component Name']),
#                 np.array(sps_data['SAP NO MA15']),
#                 np.array(sps_data['Note']),
#                 np.array(sps_data['CS'])          
#                         ]
#       index = pd.MultiIndex.from_arrays(arrays, names=('Production Module MA15', 'Step', 'SAP NO MA15', 'Note', 'CS'))
      
#       df = pd.DataFrame({'Component Name': sps_data['Component Name'].values},
#                       index=index)
      
#       # sheet_name.range('A1').options(index=True).value = df
#       df.to_excel(writer, sheet_name=sheet, index=True)

# wb.sheets[0].delete()

In [153]:
# import xlwings as xw
# import pandas as pd
# import numpy as np

# wb = xw.Book()
# sheet = wb.sheets[0]
    
# sps_data = df_input[df_input['SPS'] == "SA36-A"]
    
# arrays = [np.array(sps_data['Production Module MA15']),
#               np.array(sps_data['Step'])         
#                        ]
# index = pd.MultiIndex.from_arrays(arrays, names=('Production Module MA15', 'Step'))
    
# df = pd.DataFrame({'Component Name': sps_data['Component Name'].values},
#                      index=index)
    
# sheet.range('A1').value = df



In [154]:
import xlwings as xw
import pandas as pd
import numpy as np

all_sheets = ['SA36-A', 'SA47', 'SA513']   

with pd.ExcelWriter('schema.xlsx', engine='xlsxwriter') as writer:      
 

    for sheet in all_sheets:
        
        # Filtrage des données pour ce SPS
        sps_data = df_input[df_input['SPS'] == sheet]
        
        
        # Création du MultiIndex
        arrays = [np.array(sps_data['Production Module MA15']),
                np.array(sps_data['Step'])]
        index = pd.MultiIndex.from_arrays(arrays, names=('Production Module MA15', 'Sequence'))
        
        # Création du DataFrame avec la colonne renommée
        df = pd.DataFrame({'Materiel': sps_data['Component Name'].values,
                           'SAP NO MA15': sps_data['SAP NO MA15'].values,
                           'Note': sps_data['Note'].values,
                           'CS' : sps_data['CS'].values,
                           'color' : sps_data['Colour'].values,
                           '' : '',
                           'CON A' : sps_data['From Connector'].values,
                           'CAV A' : sps_data['From Cavity'].values,
                           'INSERTION A' : '',
                           'CON B' : sps_data['To Connector'].values,
                           'CAV B' : sps_data['To Cavity'].values,
                           'INSERTION B' : '',
                           },
                        index=index)
        
        # Écriture du DataFrame dans la feuille
        df.to_excel(writer, sheet_name=sheet, index=True)
        

In [155]:
import openpyxl
from openpyxl.styles import PatternFill, Border, Side
from openpyxl.utils import get_column_letter

color_dict = {
    'Y': 'FFFF00',  # Yellow
    'G': '00FF00',  # Green
    'L': '0000FF',  # Blue
    'R': 'FF0000',  # Red
    'W': 'FFFFFF',  # White
    'LG': '808080', # Gray (Light Gray)
    'O': 'FFA500',  # Orange
    'BR': '80471c',#'A52A2A', # Brown
    'V': '800080',#'8A2BE2',   # Violet
    'GY': '707070',#'C0C0C0',  # Gray
    'B': '000000'   # Black (Noir)
}

def apply_color(symbol, cell):
    if '/' in symbol:
        primary, secondary = symbol.split('/')
        fill = PatternFill(start_color=color_dict[primary], end_color=color_dict[primary], fill_type="solid")
        cell.fill = fill
        
        side = Side(border_style="thick", color=color_dict[secondary])
        border = Border(diagonal=side, diagonalUp=True)
        cell.border = border
    else:
        fill = PatternFill(start_color=color_dict[symbol], end_color=color_dict[symbol], fill_type="solid")
        cell.fill = fill

In [156]:
import openpyxl
from openpyxl.utils import get_column_letter

wb_path = "schema.xlsx"
wb = openpyxl.load_workbook(wb_path)

for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    
    color_col = None
    header_row = 1  
    
    for col in range(1, sheet.max_column + 1):
        if sheet.cell(row=header_row, column=col).value == "color":
            color_col = col
            break
    
    if color_col is None:
        print(f"No color column found in sheet: {sheet_name}")
        continue
    
    # Add new X column header after last column
    # x_col = sheet.max_column + 1
    # sheet.cell(row=header_row, column=x_col).value = "X"
    
    for row in range(header_row + 1, sheet.max_row + 1):
        color_symbol = sheet.cell(row=row, column=color_col).value
        if color_symbol:
            cell = sheet.cell(row=row, column=color_col + 1)
            try:
                apply_color(color_symbol, cell)
            except KeyError:
                print(f"Unknown color code: {color_symbol} in sheet {sheet_name}, row {row}")
                cell.value = "Unknown color code" #PatternFill(start_color="C0C0C0", end_color="C0C0C0", fill_type="solid")

wb.save("schema.xlsx")
print("Color formatting completed successfully.")

Color formatting completed successfully.


In [157]:
# import openpyxl
# from openpyxl.styles import Border, Side

# # Charger le fichier Excel
# workbook = openpyxl.load_workbook('schema.xlsx')
# sheet = workbook.active  # Sélectionner la première feuille (ou spécifiez le nom de la feuille)

# # Trouver la dernière ligne
# last_row = sheet.max_row +2

# # Les données à insérer
# tableau = [
#     ["PM Basique", "Note", "Les cases colorées sont des PM optionnelles", "", "", "", "", "", ""],
#     ["", "🌀 :", "A Insérer", "", "", "", "", "", ""],
#     ["", "Ø :", "A Ne pas insérer", "", "", "", "", "", ""],
#     ["", "O :", "Déjà inséré", "", "", "", "", "", ""]
# ]

# # Insérer le tableau à la ligne suivante
# for i, row in enumerate(tableau, start=last_row + 2):
#     for j, value in enumerate(row):
#         sheet.cell(row=i, column=j+1, value=value)

# # Définir la taille des colonnes
# column_widths = [15, 35, 50, 10, 10, 10, 15, 15, 15]
# for col_num, width in enumerate(column_widths, start=1):
#     sheet.column_dimensions[openpyxl.utils.get_column_letter(col_num)].width = width

# # Définir les bordures
# border = Border(
#     top=Side(style='thin'),
#     bottom=Side(style='thin'),
#     left=Side(style='thin'),
#     right=Side(style='thin')
# )

# # Appliquer les bordures à toutes les cellules
# for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
#     for cell in row:
#         cell.border = border

# # Sauvegarder le fichier
# workbook.save('votre_fichier_modifié.xlsx')


In [158]:
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, Border, Side

# Charger le fichier Excel existant
file_path = "schema.xlsx"  # Remplacez par votre chemin
wb = load_workbook(file_path)
# ws = wb.active 

for ws in wb.worksheets:
# Trouver la première ligne vide
    row_start = ws.max_row + 2  # Ajouter un espace après le tableau existant

    # Définition des données du nouveau tableau
    table_data = [
        ["", "PM Basique", "", "Niveau", "N° de Phase", "Date", "Préparé par", "Timbre"],
        ["Note:", "Les cases colorées sont des PM optionnelles", "", "", "", "", "", ""],
        ["", "🌀 : ", "A Insérer", "", "", "", "", ""],
        ["", "Ø : ", "A Ne pas insérer", "", "", "", "", ""],
        ["", "⊖ : ", "Déjà inséré", "", "", "", "", ""]
    ]

    # Définir les bordures
    border = Border(
        top=Side(style='thin'),
        bottom=Side(style='thin'),
        left=Side(style='thin'),
        right=Side(style='thin')
    )

    # Insérer les données dans la feuille
    for row_index, row_data in enumerate(table_data, start=row_start):
        for col_index, cell_value in enumerate(row_data, start=1):
            ws.cell(row=row_index, column=col_index, value=cell_value)
            ws.cell(row=row_index, column=col_index).border = border
            ws.cell(row=row_index, column=col_index).font = Font(size=12, bold=True)



        
# # Appliquer les bordures        
# for row in ws.iter_rows(min_row=row_start, max_row=row_start + len(table_data) - 1, min_col=1, max_col=len(table_data[0])):
#     for cell in row:
#         cell.border = border
        
# Sauvegarder le fichier
wb.save(file_path)
print("Tableau ajouté avec succès !")
 

Tableau ajouté avec succès !


In [159]:
import openpyxl
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils import get_column_letter

def create_yazaki_template():
    # Create a new workbook
    wb = openpyxl.Workbook()
    ws = wb.active
    
    # Set column widths
    column_widths = [15, 20, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
    for i, width in enumerate(column_widths, 1):
        ws.column_dimensions[get_column_letter(i)].width = width
    
    # Define border styles
    thin_border = Border(
        left=Side(style='thin'), 
        right=Side(style='thin'), 
        top=Side(style='thin'), 
        bottom=Side(style='thin')
    )
    
    # Header row 1 - Yazaki logo area
    ws.merge_cells('A1:O1')
    ws['A1'] = '../yazaki_logo.png'
    ws['A1'].font = Font(name='Arial', size=14, bold=True)
    ws['A1'].alignment = Alignment(horizontal='left', vertical='center')
    
    # Header row 2 - Product Details
    header_fill = PatternFill(start_color='D9D9D9', end_color='D9D9D9', fill_type='solid')
    header_font = Font(name='Arial', size=10)
    
    header_cells = [
        ('A2', 'N° du Produit / Niveau'),
        ('B2', ''),
        ('C2', 'Local du travail'),
        ('D2', 'WP414-SA513'),
        ('E2', 'N° de ligne'),
        ('F2', '1'),
        ('G2', 'Processus'),
        ('H2', 'C2'),
        ('I2', 'PU24/PU25 LHD'),
        ('J2', ''),
        ('K2', 'N° de Registre'),
        ('L2', 'EA-EN-MM0.x.1-6047')
    ]
    
    for cell, value in header_cells:
        ws[cell] = value
        ws[cell].font = header_font
        ws[cell].alignment = Alignment(horizontal='center', vertical='center')
        ws[cell].border = thin_border
        ws[cell].fill = header_fill
    
    # Product Number specifics
    ws['A2'].font = Font(name='Arial', size=10, bold=True)
    ws['A2'].value = 'KAR G60'
    
    # Save the workbook
    wb.save('Yazaki_Template.xlsx')
    print("Yazaki template created successfully!")

# Run the function
create_yazaki_template()

Yazaki template created successfully!


In [171]:
import openpyxl
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill, Color
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image

def create_yazaki_template():
    # Create a new workbook
    wb = openpyxl.Workbook()
    ws = wb.active
    
    # Set column widths
    column_widths = [15, 20, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
    for i, width in enumerate(column_widths, 1):
        ws.column_dimensions[get_column_letter(i)].width = width
    
    # Define border styles
    thin_border = Border(
        left=Side(style='thin'), 
        right=Side(style='thin'), 
        top=Side(style='thin'), 
        bottom=Side(style='thin')
    )
    
    # Header row 1 - Yazaki logo area
    img = Image('../yazaki_logo.png')
    cell_logo = ws.merge_cells('A1:J1')
    ws.add_image(img, cell_logo)
    # ws['A1'] = 'YAZAKI'
    # ws['A1'].font = Font(name='Arial', size=14, bold=True)
    # ws['A1'].alignment = Alignment(horizontal='left', vertical='center')
    
    # Blue color for PU24/PU25 LHD
    blue_fill = PatternFill(start_color='0070C0', end_color='0070C0', fill_type='solid')
    blue_font = Font(name='Arial', size=10, color=Color(rgb='FFFFFF'))
    
    # Merge and style PU24/PU25 LHD
    ws.merge_cells('K1:O1')
    # Get the first cell of the active sheet to set the value
    first_cell = ws.cell(row=1, column=11)  # Column K is the 11th column
    first_cell.value = 'PU24/PU25 LHD'
    first_cell.font = blue_font
    first_cell.fill = blue_fill
    first_cell.alignment = Alignment(horizontal='center', vertical='center')
    
    # Header row 2 and 3 with specific requirements
    header_fill = PatternFill(start_color='D9D9D9', end_color='D9D9D9', fill_type='solid')
    header_font = Font(name='Arial', size=10)
    
    # Specific cell configurations
    cell_configs = [
        ('C2', 'Nº du Produit / Niveau', header_font, header_fill, thin_border),
        ('C3', 'KAR G60', Font(name='Arial', size=10), None, thin_border),
        ('E2', 'Local du travail', header_font, header_fill, thin_border),
        ('E3', 'WP414-SA513', Font(name='Arial', size=10), None, thin_border),
        ('F2', 'N° de ligne', header_font, header_fill, thin_border),
        ('F3', '1', Font(name='Arial', size=10), None, thin_border),
        ('G2', 'N° de ligne', header_font, header_fill, thin_border),
        ('G3', '1', Font(name='Arial', size=10), None, thin_border),
        ('H2', 'N° de ligne', header_font, header_fill, thin_border),
        ('H3', '1', Font(name='Arial', size=10), None, thin_border),
        ('I2:J2', 'Processus', header_font, header_fill, thin_border),
        ('I3:J3', 'C2', Font(name='Arial', size=10), None, thin_border),
        ('M2', 'Nº de Registre', header_font, header_fill, thin_border),
        ('M3', 'EA-EN-MMO-xx-T-6047', Font(name='Arial', size=10), None, thin_border)
    ]
    
    for cell_range, value, font, fill, border in cell_configs:
        # Handle merged cells
        if ':' in cell_range:
            ws.merge_cells(cell_range)
            # Get the first cell of the merged range
            first_cell = ws[cell_range.split(':')[0]]
            first_cell.value = value
            first_cell.font = font
            
            if fill:
                first_cell.fill = fill
            
            first_cell.border = border
            first_cell.alignment = Alignment(horizontal='center', vertical='center')
        else:
            cell = ws[cell_range]
            cell.value = value
            cell.font = font
            
            if fill:
                cell.fill = fill
            
            cell.border = border
            cell.alignment = Alignment(horizontal='center', vertical='center')
    
    # Add borders to entire merged ranges
    border_ranges = ['C2:D2', 'I2:J2', 'M2:O2', 'C3:D3', 'I3:J3', 'M3:O3']
    for cell_range in border_ranges:
        for row in ws[cell_range]:
            for cell in row:
                cell.border = thin_border
    
    # Save the workbook
    wb.save('Yazaki_Template.xlsx')
    print("Yazaki template created successfully!")

# Run the function
create_yazaki_template()

Yazaki template created successfully!
